In [ ]:
import html
import nltk
import os
import re
import sys

In [ ]:
module_path = os.path.abspath(os.path.join('../embedding-classif/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
os.environ["CLASSPATH"] = "/home/felipe/Downloads/stanford"

In [ ]:
from helpers import files_helper

In [ ]:
titles,bodies, labels = files_helper.read_stackoverflow_sample_small(join=False)

In [ ]:
i = 3

title,body=titles[i], bodies[i]

In [ ]:
title,body

In [ ]:
def tokenizePost(title, body, unescapeXml = False):
    # tokenize the string the same way I tokenized the whole dataset in
    # https://github.com/queirozfcom/auto-tagger/blob/master/scala/autotagger-spark-2/src/main/scala/ReadSOStanfordTokenize.scala
    
    HTML_TAGS_PATTERN = re.compile("""<[^>]+>""")
    WHITESPACE_OR_NEWLINE_PATTERN = re.compile("""\s+""")
    FORWARD_SLASH_PATTERN = re.compile("""((?<!\d)/(?!\d))""") # slash not preceded a digit, not followed by a digit
    DOT_PATTERN = re.compile("""((?!<=\d)\.(?!\d))""")
    TRIPLE_DOTS_PATTERN = re.compile("""\.\s\.\s\.""")
    
    OPTIONS = {"ptb3Escaping":False}
    
    titleOut = ""
    bodyOut = ""
    
    if unescapeXml:
        titleOut = html.unescape(title).lower()
        bodyOut = html.unescape(body).lower()
    else:
        titleOut = title.lower()
        bodyOut = body.lower()        
    
    # this is done to separate title from body with a period, if non exists
    if titleOut.endswith("?") or titleOut.endswith("!") or titleOut.endswith("."):
        pass # do nothing
    else:
        titleOut = titleOut + "."
    
    bodyOut = HTML_TAGS_PATTERN.sub(" ", bodyOut) # take out htmltags
    bodyOut = FORWARD_SLASH_PATTERN.sub(" / ", bodyOut) # otherwise the tokenizer doesn't consider a "/" as a delimiter
    bodyOut = DOT_PATTERN.sub(" . ", bodyOut) # to make all dots be delimiters, not just some (except in numbers)
    bodyOut = WHITESPACE_OR_NEWLINE_PATTERN.sub(" ", bodyOut) # replace multiple whitespaces/newlines with a single one
        
    combinedOut = titleOut + " " + bodyOut
    
    tokenizer = StanfordTokenizer(options=OPTIONS)
    
    tokens = tokenizer.tokenize(combinedOut)
      
    tokenizedOut = " ".join(tokens)    

    # after tokenizing, remove spaces between triple dots
    
    tokenizedOut = re.sub(TRIPLE_DOTS_PATTERN,"...",tokenizedOut)
    tokenizedOut = re.sub(WHITESPACE_OR_NEWLINE_PATTERN, " ",tokenizedOut)
    
    return tokenizedOut.strip()


tokenizePost(title,body,False)